# 01 | Transform the Data

# Introduction
In this notebook, we will deal with the data transformation steps required to proceed with our analysis. This process will be reproduced as .py files later on.

# Goals:
Targeted objectives in this notebook ar checked as follows:

- [ ] Import the raw data
  -  [ ] SIPRI dataset and capitals coordinates
- [ ]  Store the raw data
- [x] Prepare the data
  - [x] Clean each individual table
  - [x] Store the transformed dataset
- [ ] Combine both datasets
  - Check if the merging column values match
- [ ] Store the final output

# Set up our working environment

In [556]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from geopy.distance import geodesic
from tableauhyperapi import (
    HyperProcess,
    Telemetry,
    Connection,
    CreateMode,
    NOT_NULLABLE,
    NULLABLE,
    SqlType,
    TableDefinition,
    Inserter,
    HyperException,
    )

In [557]:
# Create directory folders to store our data
dirname = os.getcwd()

raw_data = f"{dirname}/data/raw/"
transformed_data = f"{dirname}/data/transformed/"
refined_data = f"{dirname}/data/refined/"

paths = [raw_data, transformed_data, refined_data]

for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)

# Data Transformation | SIPRI dataset
Let's start by preparing our main dataset for data analysis.

## Import raw data
We'll first import the raw data that we stored during the previous step. Because we have three indivual tables that must turn into one, we must establish a few steps:
- Import a single table;
- Analyse the required transformation steps we need;
- Apply the transformations;
- Loop through the 3 tables and pass the same steps (if possible);

### Import data
We can start by using table 5 (Constant US$ (2022)). 

In [558]:
sipri_raw = sipri_raw = pd.read_csv(f"{raw_data}sipri_data_raw_tb_5.csv")
sipri_raw.head()

,"Military expenditure by country, in millions of US$ at current prices and exchange rates, 1948-2023 © SIPRI 2023",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
0,"Figures are in US $m., in current prices, conv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Figures in blue are SIPRI estimates. Figures i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,""". ."" = data unavailable. ""xxx"" = country did ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Country,Notes,1948,1949,1950,1951,1952,1953,1954,1955,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023


**Challenge** – the table contains additional non-tabular information on the first rows. We must remove them. Considering that the number of extra rows may differ between each `.csv` file, we need to dynamically identify what is our header. We need to:
- Read the file.
- Iterate through each line using `enumerate`.
- For each value in each index, if `Country` appears, return the index value `i` and define it as `header_row`
  - It will be used to define the number of rows we need to skip when reading our `.csv` file.

In [559]:
# Read the file in chunks to find the header row
with open(f"{raw_data}sipri_data_raw_tb_5.csv", "r") as file:
    for i, line in enumerate(file):
        if "Country" in line:
            header_row = i # define header if `Country` is a value
            break

In [560]:
sipri_raw_tb5 = pd.read_csv(f"{raw_data}sipri_data_raw_tb_5.csv", skiprows=header_row, header=0)
sipri_raw_tb5.head()

,Country,Notes,1948,1949,1950,1951,1952,1953,1954,1955,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,North Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,§,...,...,...,...,...,...,...,...,...,9724.379971923256,10412.714002896393,10217.081699569308,10073.364021301344,9583.7242883703,10303.60057521065,9708.277440227255,9112.461105348943,9145.810174207281,18263.96796826213
4,Libya,‡§¶,...,...,...,...,...,...,...,...,...,3755.652496350929,...,...,...,...,...,...,...,...,...


In [561]:
sipri_raw_tb5.tail()

,Country,Notes,1948,1949,1950,1951,1952,1953,1954,1955,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
188,Syria,§,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Türkiye,‖,...,197.68186020052622,212.97020550380432,231.81397994737964,257.7686126715495,294.0339899025812,332.0770817037616,382.9197184100121,...,17576.538470505897,15668.749999999998,17827.702150796664,17822.738263164494,19648.69382385138,20436.917121238785,17478.41368526983,15567.410029425082,10779.896284618242,15827.853255045886
190,United Arab Emirates,§,...,...,...,...,...,...,...,...,...,22755.071477195375,...,...,...,...,...,...,...,...,...
191,"Yemen, North",§,...,...,...,...,...,...,...,...,...,xxx,xxx,xxx,xxx,xxx,xxx,xxx,xxx,xxx,xxx
192,Yemen,§,...,...,...,...,...,...,...,...,...,1714.8308436874681,...,...,...,...,...,...,...,...,...


Great! Now, we can take a brief look at the structure of our dataset.

In [562]:
sipri_raw_tb5.shape

(193, 78)

In [563]:
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 78 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  192 non-null    object
 1   Notes    79 non-null     object
 2   1948     174 non-null    object
 3   1949     174 non-null    object
 4   1950     174 non-null    object
 5   1951     174 non-null    object
 6   1952     174 non-null    object
 7   1953     174 non-null    object
 8   1954     174 non-null    object
 9   1955     174 non-null    object
 10  1956     174 non-null    object
 11  1957     174 non-null    object
 12  1958     174 non-null    object
 13  1959     174 non-null    object
 14  1960     174 non-null    object
 15  1961     174 non-null    object
 16  1962     174 non-null    object
 17  1963     174 non-null    object
 18  1964     174 non-null    object
 19  1965     174 non-null    object
 20  1966     174 non-null    object
 21  1967     174 non-null    object
 22  19

In [564]:
sipri_raw_tb5["Country"].unique()

array([nan, 'Africa', 'North Africa', 'Algeria', 'Libya', 'Morocco',
       'Tunisia', 'sub-Saharan Africa', 'Angola', 'Benin', 'Botswana',
       'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde',
       'Central African Republic', 'Chad', 'Congo, DR', 'Congo, Republic',
       "Cote d'Ivoire", 'Djibouti', 'Equatorial Guinea', 'Eritrea',
       'Ethiopia', 'Gabon', 'Gambia, The', 'Ghana', 'Guinea',
       'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Madagascar',
       'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Mozambique',
       'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'Seychelles',
       'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan',
       'Eswatini', 'Tanzania', 'Togo', 'Uganda', 'Zambia', 'Zimbabwe',
       'Americas', 'Central America and the Caribbean', 'Belize',
       'Costa Rica', 'Cuba', 'Dominican Republic', 'El Salvador',
       'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua',
       'Panama', 'Trinidad and Toba

After a quick glance, we've identified a few cleaning tasks:
- Remove unwanted columns
- Pivot year columns
- Remove regions
- Filter data
- Define each data type for each column
- Rename country names (if needed)
- Rename column names


We need to perform these tasks for each table. Then, we need to
- [ ] Join the three tables
  
### Remove unwanted columns
Remove `Notes`.

In [565]:
columns_to_drop = sipri_raw_tb5.columns[1]
sipri_raw_tb5 = sipri_raw_tb5.drop(columns=columns_to_drop)

In [566]:
sipri_raw_tb5.head()

,Country,1948,1949,1950,1951,1952,1953,1954,1955,1956,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,North Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,...,...,...,...,...,...,...,...,...,...,9724.379971923256,10412.714002896393,10217.081699569308,10073.364021301344,9583.7242883703,10303.60057521065,9708.277440227255,9112.461105348943,9145.810174207281,18263.96796826213
4,Libya,...,...,...,...,...,...,...,...,...,...,3755.652496350929,...,...,...,...,...,...,...,...,...


### Pivot Year Columns

In [567]:
columns_to_pivot = sipri_raw_tb5.columns[1:]
sipri_raw_tb5 = sipri_raw_tb5.melt(
    id_vars="Country",
    value_vars=columns_to_pivot,
    var_name="year",
    value_name="military_expenditures_usd_2022")

In [568]:
sipri_raw_tb5

,Country,year,military_expenditures_usd_2022
0,NaN,1948,NaN
1,Africa,1948,NaN
2,North Africa,1948,NaN
3,Algeria,1948,...
4,Libya,1948,...
...,...,...,...
14663,Syria,2023,...
14664,Türkiye,2023,15827.853255045886
14665,United Arab Emirates,2023,...
14666,"Yemen, North",2023,xxx


### Remove regions
We can perform this task by cleaning null values from our pivoted column.

In [569]:
sipri_raw_tb5 = sipri_raw_tb5.dropna(subset=["military_expenditures_usd_2022"])

In [570]:
sipri_raw_tb5

,Country,year,military_expenditures_usd_2022
3,Algeria,1948,...
4,Libya,1948,...
5,Morocco,1948,...
6,Tunisia,1948,...
8,Angola,1948,...
...,...,...,...
14663,Syria,2023,...
14664,Türkiye,2023,15827.853255045886
14665,United Arab Emirates,2023,...
14666,"Yemen, North",2023,xxx


In [571]:
# Check for empty rows
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13224 entries, 3 to 14667
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country                         13224 non-null  object
 1   year                            13224 non-null  object
 2   military_expenditures_usd_2022  13224 non-null  object
dtypes: object(3)
memory usage: 413.2+ KB


### Define data types
We need to:
- Convert `year` to `int`.
- Convert `military_expenditures_usd_2022` to `float`.
  - First, convert missing values signed as `xxx` to `NaN`.

In [572]:
# Identify non-numeric values
non_numeric_rows = sipri_raw_tb5[pd.to_numeric(sipri_raw_tb5["military_expenditures_usd_2022"], errors="coerce").isna()]
non_numeric_rows

,Country,year,military_expenditures_usd_2022
3,Algeria,1948,...
4,Libya,1948,...
5,Morocco,1948,...
6,Tunisia,1948,...
8,Angola,1948,...
...,...,...,...
14661,Qatar,2023,...
14663,Syria,2023,...
14665,United Arab Emirates,2023,...
14666,"Yemen, North",2023,xxx


In [573]:
non_numeric_rows["military_expenditures_usd_2022"].unique()

array(['...', 'xxx'], dtype=object)

Ok! We've identified our null values. We need to replace them with NaN and convert the column to numeric.

In [574]:
# Replace non-numeric values with NaN
sipri_raw_tb5.loc[:,"military_expenditures_usd_2022"] = pd.to_numeric(
    sipri_raw_tb5["military_expenditures_usd_2022"], errors="coerce"
)

In [575]:
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13224 entries, 3 to 14667
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country                         13224 non-null  object
 1   year                            13224 non-null  object
 2   military_expenditures_usd_2022  8697 non-null   object
dtypes: object(3)
memory usage: 413.2+ KB


We can now remove NaN values.

In [576]:
sipri_raw_tb5 = sipri_raw_tb5.dropna(subset=["military_expenditures_usd_2022"])
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8697 entries, 58 to 14664
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country                         8697 non-null   object
 1   year                            8697 non-null   object
 2   military_expenditures_usd_2022  8697 non-null   object
dtypes: object(3)
memory usage: 271.8+ KB


Great! Let's move on. We now need to convert `year` to `int`.

In [577]:
sipri_raw_tb5 = sipri_raw_tb5.copy()
sipri_raw_tb5["year"] = sipri_raw_tb5["year"].astype(int)
sipri_raw_tb5["military_expenditures_usd_2022"] = sipri_raw_tb5["military_expenditures_usd_2022"].astype(float)

In [578]:
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8697 entries, 58 to 14664
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         8697 non-null   object 
 1   year                            8697 non-null   int64  
 2   military_expenditures_usd_2022  8697 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 271.8+ KB


### Filter data
We need to get:
- Data from 1990 onwards
- Limit our request to European countries only.

Let's start with the first task.

In [579]:
# Get years from 1990 to 2024
years = sipri_raw_tb5["year"].astype(int)
years = years[(years >= 1993) & (years <= 2024)]
years = years.unique()
years



array([1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [580]:
# Filter the dataset using the `years` variable as filter
sipri_raw_tb5 = sipri_raw_tb5[sipri_raw_tb5["year"].isin(years)]
sipri_raw_tb5

,Country,year,military_expenditures_usd_2022
8688,Algeria,1993,1276.911083
8690,Morocco,1993,1251.786538
8691,Tunisia,1993,276.167135
8693,Angola,1993,1774.397877
8695,Botswana,1993,185.671483
...,...,...,...
14658,Kuwait,2023,7755.031936
14659,Lebanon,2023,241.290151
14660,Oman,2023,5851.755527
14662,Saudi Arabia,2023,75813.333333


In [581]:
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4749 entries, 8688 to 14664
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         4749 non-null   object 
 1   year                            4749 non-null   int64  
 2   military_expenditures_usd_2022  4749 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 148.4+ KB


Let's now get a list of countries and return only a list of European countries.

In [582]:
sipri_raw_tb5["Country"].sort_values().unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Colombia', 'Congo, DR', 'Congo, Republic',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'European Union', 'Fiji', 'Finland',
       'France', 'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana',
       'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
       'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia',
       'Iran', 'Iraq', 'Irel

In [583]:
# List European countries
european_countries = [
    "Albania", "Armenia", "Austria", "Azerbaijan", 
    "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria",
    "Croatia", "Cyprus", "Czechia", "Czechoslovakia", "Denmark",
    "Estonia", "Finland", "France", "Georgia",
    "Germany", "Greece", "Hungary", "Iceland", 
    "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania",
    "Luxembourg", "Malta", "Moldova", "Montenegro",
    "Netherlands", "North Macedonia", "Norway", "Poland",
    "Portugal", "Romania", "Russia", "Serbia", 
    "Slovakia", "Slovenia", "Spain", "Sweden",
    "Switzerland", "Türkiye", "USSR", "Ukraine",
    "United Kingdom", "Yugoslavia"
]

sipri_raw_tb5 = sipri_raw_tb5[sipri_raw_tb5["Country"].isin(european_countries)]
sipri_raw_tb5

,Country,year,military_expenditures_usd_2022
8810,Albania,1993,39.289843
8812,Bulgaria,1993,282.674243
8813,Croatia,1993,577.978269
8814,Czechia,1993,815.585467
8816,Estonia,1993,13.146560
...,...,...,...
14646,Spain,2023,23699.130514
14647,Sweden,2023,8754.872951
14648,Switzerland,2023,6293.390648
14649,United Kingdom,2023,74942.843460


In [584]:
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1349 entries, 8810 to 14664
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         1349 non-null   object 
 1   year                            1349 non-null   int64  
 2   military_expenditures_usd_2022  1349 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 42.2+ KB


Great! Now, our values are written in millions but not literally - we need to multiply each row value by `1_000_000`

In [585]:
sipri_raw_tb5["military_expenditures_usd_2022"] = sipri_raw_tb5["military_expenditures_usd_2022"] * 1_000_000
sipri_raw_tb5

C:\Users\felip\AppData\Local\Temp\ipykernel_22168\2468274360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sipri_raw_tb5["military_expenditures_usd_2022"] = sipri_raw_tb5["military_expenditures_usd_2022"] * 1_000_000


,Country,year,military_expenditures_usd_2022
8810,Albania,1993,3.928984e+07
8812,Bulgaria,1993,2.826742e+08
8813,Croatia,1993,5.779783e+08
8814,Czechia,1993,8.155855e+08
8816,Estonia,1993,1.314656e+07
...,...,...,...
14646,Spain,2023,2.369913e+10
14647,Sweden,2023,8.754873e+09
14648,Switzerland,2023,6.293391e+09
14649,United Kingdom,2023,7.494284e+10


Awesome! Now, we just need to rename our `Country` column to `country`.

In [586]:
sipri_raw_tb5 = sipri_raw_tb5.rename(columns={
    "Country": "country"
})

In [587]:
sipri_raw_tb5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1349 entries, 8810 to 14664
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         1349 non-null   object 
 1   year                            1349 non-null   int64  
 2   military_expenditures_usd_2022  1349 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 42.2+ KB


In [588]:
sipri_raw_tb5.describe()

,year,military_expenditures_usd_2022
count,1349.000000,1.349000e+03
mean,2008.350630,7.404347e+09
std,8.861494,1.483012e+10
min,1993.000000,0.000000e+00
25%,2001.000000,2.878641e+08
50%,2009.000000,1.403134e+09
75%,2016.000000,5.383117e+09
max,2023.000000,1.094544e+11


Awesome! Our table is ready. We need to perform these same tasks on tables #6 and #7. It would be the right time to create a few functions to automate this process.

In [589]:
def transform_data(raw_data, table_number, column_name, start_year, end_year, countries_list, renamed_column) -> pd.DataFrame:
    """
    Reads the `.csv` file and performs a series of transformations.
    """
    # Initialize header_row
    header_row = None

    # Debug: Print the table being processed
    print(f"Processing table {table_number}...")

    # Dynamically find the header row
    with open(f"{raw_data}sipri_data_raw_tb_{table_number}.csv", "r") as file:
        for i, line in enumerate(file):
            if "Country" in line:
                header_row = i  # Define header if `Country` is a value
                break

    # Debug: Print the header_row value
    print(f"Header row for table {table_number}: {header_row}")

    # Raise an error if header_row is not found
    if header_row is None:
        raise ValueError(f"'Country' header not found in table {table_number}")

    # Read the file
    df = pd.read_csv(f"{raw_data}sipri_data_raw_tb_{table_number}.csv", skiprows=header_row, header=0)

    print(f"\nFirst 3 rows: \n{df.head(3)}")
    
    # Dynamically identify columns to drop
    # Keep "Country" and all columns starting with a year
    columns_to_keep = ["Country"] + [col for col in df.columns if col.strip().isdigit()]
    df = df[columns_to_keep]
    
    # Pivot year columns
    columns_to_pivot = df.columns[1:]
    df = df.melt(
        id_vars="Country",
        value_vars=columns_to_pivot,
        var_name="year",
        value_name=column_name
    )

    # Remove regions
    df = df.dropna(subset=[column_name])

    # Replace non-numeric values with NaN
    df.loc[:, column_name] = pd.to_numeric(df[column_name], errors="coerce")

    # Convert data types to numeric
    df["year"] = df["year"].astype(int)
    df[column_name] = df[column_name].astype(float)

    # Filter the dataset by years
    df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

    # Filter by European countries
    df = df[df["Country"].isin(countries_list)]

    # If table is 5 or 6, multiply column value by 1,000,000
    if table_number in [4, 5]:
        df[column_name] *= 1_000_000

    # Rename `Country` column
    df = df.rename(columns={
    "Country": renamed_column
})

    print(f"Table {table_number} output: \n{df.head()}")
    return df

In [590]:
# Define the parameters for each table
tables = [
    {"table_number" : 4, "column_name": "military_expenditures_usd_2022"},
    {"table_number" : 5, "column_name": "military_expenditures_current_usd"},
    {"table_number" : 6, "column_name": "share_GDP"},
    {"table_number" : 8, "column_name": "share_govt"},
]

start_year = 1993
end_year = 2024

countries_list = [
    "Albania", "Armenia", "Austria", "Azerbaijan", 
    "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria",
    "Croatia", "Czechia", "Czechoslovakia", "Denmark",
    "Estonia", "Finland", "France", "Georgia",
    "Germany", "Greece", "Hungary", "Iceland", 
    "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania",
    "Luxembourg", "Malta", "Moldova", "Montenegro",
    "Netherlands", "North Macedonia", "Norway", "Poland",
    "Portugal", "Romania", "Russia", "Serbia", 
    "Slovakia", "Slovenia", "Spain", "Sweden",
    "Switzerland", "Türkiye", "USSR", "Ukraine",
    "United Kingdom", "Yugoslavia"
]

renamed_column = "country"

In [591]:
# Store the resulting DataFrames in variables
sipri_tb4, sipri_tb5, sipri_tb6, sipri_tb8 = [
    transform_data(
        raw_data=raw_data,
        table_number=table["table_number"],
        column_name=table["column_name"],
        start_year=start_year,
        end_year=end_year,
        countries_list=countries_list,
        renamed_column=renamed_column
    )
for table in tables]

Processing table 4...
Header row for table 4: 5

First 3 rows: 
        Country  Unnamed: 1 Notes 1948 1949 1950 1951 1952 1953 1954  ...  \
0           NaN         NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   
1        Africa         NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   
2  North Africa         NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   

  2014 2015 2016 2017 2018 2019 2020 2021 2022 2023  
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[3 rows x 79 columns]
Table 4 output: 
                     country  year  military_expenditures_usd_2022
8810                 Albania  1993                    1.515335e+08
8811  Bosnia and Herzegovina  1993                             NaN
8812                Bulgaria  1993                    1.134818e+09
8813                 Croatia  1993                    2.609599e+09
8814                 Czechia  1993  

Ok! Let's move on to the next part.

## Merge all tables
We will now merge all the tables and get the following structure:
| country    | year | military_expenditures_usd_2022  | military_expenditures_current_usd | share_GDP | share_govt | 
| -------- | ------- | ------- | ------- | ------- | ------- |


In [592]:
sipri_joined_table = sipri_tb4.merge(sipri_tb5, on=["country", "year"],how="outer") \
                              .merge(sipri_tb6, on=["country", "year"], how="outer") \
                              .merge(sipri_tb8, on=["country", "year"], how="outer") 

sipri_joined_table.head()   


,country,year,military_expenditures_usd_2022,military_expenditures_current_usd,share_GDP,share_govt
0,Albania,1993,1.515335e+08,3.928984e+07,0.031995,NaN
1,Albania,1994,1.448474e+08,4.964950e+07,0.025478,NaN
2,Albania,1995,1.349759e+08,5.090752e+07,0.020536,NaN
3,Albania,1996,1.212103e+08,4.571336e+07,0.013790,NaN
4,Albania,1997,8.462970e+07,2.982549e+07,0.012831,0.044247


In [593]:
# Sample data from a single country.
sipri_joined_table[sipri_joined_table["country"]=="France"]

,country,year,military_expenditures_usd_2022,military_expenditures_current_usd,share_GDP,share_govt
434,France,1993,5.001899e+10,3.577527e+10,0.026895,0.048988
435,France,1994,5.027632e+10,3.728861e+10,0.026604,0.048970
436,France,1995,4.778746e+10,4.012402e+10,0.024925,0.045726
437,France,1996,4.665076e+10,3.897773e+10,0.024145,0.044230
438,France,1997,4.681931e+10,3.469790e+10,0.023754,0.043796
439,France,1998,4.557486e+10,3.363356e+10,0.022262,0.042278
440,France,1999,4.595824e+10,3.267271e+10,0.021778,0.041592
441,France,2000,4.543840e+10,2.840314e+10,0.020850,0.040366
442,France,2001,4.529832e+10,2.795157e+10,0.020307,0.039265
443,France,2002,4.622902e+10,3.057840e+10,0.020463,0.038758


Awesome! We can now export our `SIPRI` dataset to the `transformed` folder.

In [594]:
# Export output to transformed
sipri_joined_table.to_csv(f"{transformed_data}sipri_dataset.csv", index=False)

# Data transformation | Coordinates for City Capitals
Our goals here are:
- Import the raw data.
- Check its structure.
- Select targeted countries (European).
- Rename variables and values.
  - Make sure that names of countries matches those from SIPRI dataset.
- Create columns
  - Distance, in km and miles, from the city capital to Moscow.
## Import data

In [595]:
city_coords = pd.read_csv(f"{raw_data}city_capitals_coords.csv")

In [596]:
city_coords

,Country,Capital City,Latitude,Longitude,Population,Capital Type
0,Afghanistan,Kabul,34.5289,69.1725,4011770,Capital
1,Albania,Tiranë (Tirana),41.3275,19.8189,475577,Capital
2,Algeria,El Djazaïr (Algiers),36.7525,3.0420,2693542,Capital
3,American Samoa,Pago Pago,-14.2781,-170.7025,48526,Capital
4,Andorra,Andorra la Vella,42.5078,1.5211,22614,Capital
...,...,...,...,...,...,...
229,Wallis and Futuna Islands,Matu-Utu,-13.2816,-176.1745,1025,Capital
230,Western Sahara,El Aaiún,27.1532,-13.2014,232388,Capital
231,Yemen,Sana'a',15.3531,44.2078,2779317,Capital
232,Zambia,Lusaka,-15.4134,28.2771,2523844,Capital


## Check the table structure

In [597]:
city_coords.shape

(234, 6)

In [598]:
city_coords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country       234 non-null    object 
 1   Capital City  234 non-null    object 
 2   Latitude      234 non-null    float64
 3   Longitude     234 non-null    float64
 4   Population    234 non-null    int64  
 5   Capital Type  234 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 11.1+ KB


There is not much to do here. Our table seems to be in good shape. We can focus on:
- Selecting our countries
- Removing unnecessary columns

## Select countries, remove columns

In [599]:
city_coords["Country"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Caribbean Netherlands', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'China, Hong Kong SAR', 'China, Macao SAR',
       'China, Taiwan Province of China', 'Colombia', 'Comoros', 'Congo',
       'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba',
       'Curaçao', 'Cyprus', 'Czechia', "Dem. People's Republic of Korea",
       'Democratic Republ

In [600]:
city_coords_countries_list = [
    "Albania", "Armenia", "Austria", "Azerbaijan", 
    "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria",
    "Croatia", "Czechia", "Cyprus", "Denmark",
    "Estonia", "Finland", "France", "Georgia",
    "Germany", "Greece", "Hungary", "Iceland", 
    "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania",
    "Luxembourg", "Malta", "Republic of Moldova", "Montenegro",
    "Netherlands", "North Macedonia", "Norway", "Poland",
    "Portugal", "Romania", "Russian Federation", "Serbia", 
    "Slovakia", "Slovenia", "Spain", "Sweden", "TFYR Macedonia",
    "Switzerland", "Turkey", "Ukraine",
    "United Kingdom", "Yugoslavia"
]

In [601]:
# Select countries using list
city_coords = city_coords[city_coords["Country"].isin(city_coords_countries_list)].reset_index(drop=True)

In [602]:
city_coords

,Country,Capital City,Latitude,Longitude,Population,Capital Type
0,Albania,Tiranë (Tirana),41.3275,19.8189,475577,Capital
1,Armenia,Yerevan,40.1820,44.5146,1080324,Capital
2,Austria,Wien (Vienna),48.2064,16.3707,1900547,Capital
3,Azerbaijan,Baku,40.3777,49.8920,2285729,Capital
4,Belarus,Minsk,53.9000,27.5667,2004672,Capital
5,Belgium,Bruxelles-Brussel,50.8467,4.3499,2049510,Capital
6,Bosnia and Herzegovina,Sarajevo,43.8486,18.3564,342577,Capital
7,Bulgaria,Sofia,42.6975,23.3242,1272418,Capital
8,Croatia,Zagreb,45.8144,15.9780,685587,Capital
9,Cyprus,Lefkosia (Nicosia),35.1595,33.3669,269469,Capital


In [603]:
city_coords

,Country,Capital City,Latitude,Longitude,Population,Capital Type
0,Albania,Tiranë (Tirana),41.3275,19.8189,475577,Capital
1,Armenia,Yerevan,40.1820,44.5146,1080324,Capital
2,Austria,Wien (Vienna),48.2064,16.3707,1900547,Capital
3,Azerbaijan,Baku,40.3777,49.8920,2285729,Capital
4,Belarus,Minsk,53.9000,27.5667,2004672,Capital
5,Belgium,Bruxelles-Brussel,50.8467,4.3499,2049510,Capital
6,Bosnia and Herzegovina,Sarajevo,43.8486,18.3564,342577,Capital
7,Bulgaria,Sofia,42.6975,23.3242,1272418,Capital
8,Croatia,Zagreb,45.8144,15.9780,685587,Capital
9,Cyprus,Lefkosia (Nicosia),35.1595,33.3669,269469,Capital


In [604]:
# Remove columns
city_coords = city_coords.iloc[:, :-2]

In [605]:
city_coords

,Country,Capital City,Latitude,Longitude
0,Albania,Tiranë (Tirana),41.3275,19.8189
1,Armenia,Yerevan,40.1820,44.5146
2,Austria,Wien (Vienna),48.2064,16.3707
3,Azerbaijan,Baku,40.3777,49.8920
4,Belarus,Minsk,53.9000,27.5667
5,Belgium,Bruxelles-Brussel,50.8467,4.3499
6,Bosnia and Herzegovina,Sarajevo,43.8486,18.3564
7,Bulgaria,Sofia,42.6975,23.3242
8,Croatia,Zagreb,45.8144,15.9780
9,Cyprus,Lefkosia (Nicosia),35.1595,33.3669


## Rename variables and values
Next, we need to:
- Rename country names (when necessary)
- Rename capital names (make them shorter)
- Rename column names

### Rename country names

In [606]:
sipri_joined_table["country"].unique()

array(['Albania', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus',
       'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia',
       'Czechia', 'Czechoslovakia', 'Denmark', 'Estonia', 'Finland',
       'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland',
       'Ireland', 'Italy', 'Kosovo', 'Latvia', 'Lithuania', 'Luxembourg',
       'Malta', 'Moldova', 'Montenegro', 'Netherlands', 'North Macedonia',
       'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'Serbia',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'Türkiye', 'USSR', 'Ukraine', 'United Kingdom', 'Yugoslavia'],
      dtype=object)

In [607]:
city_coords["Country"].unique()

array(['Albania', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus',
       'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France',
       'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
       'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
       'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal',
       'Republic of Moldova', 'Romania', 'Russian Federation', 'Serbia',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'TFYR Macedonia', 'Turkey', 'Ukraine', 'United Kingdom'],
      dtype=object)

In [608]:
city_coords = city_coords.replace(
    {"Country": {
        "Republic of Moldova": "Moldova",
        "Russian Federation": "Russia",
        "Turkey": "Türkiye",
        "TFYR Macedonia": "North Macedonia",
    }}
)

In [609]:
city_coords

,Country,Capital City,Latitude,Longitude
0,Albania,Tiranë (Tirana),41.3275,19.8189
1,Armenia,Yerevan,40.1820,44.5146
2,Austria,Wien (Vienna),48.2064,16.3707
3,Azerbaijan,Baku,40.3777,49.8920
4,Belarus,Minsk,53.9000,27.5667
5,Belgium,Bruxelles-Brussel,50.8467,4.3499
6,Bosnia and Herzegovina,Sarajevo,43.8486,18.3564
7,Bulgaria,Sofia,42.6975,23.3242
8,Croatia,Zagreb,45.8144,15.9780
9,Cyprus,Lefkosia (Nicosia),35.1595,33.3669


### Format city names
Next, we keep only the values in English when possible.

In [610]:
city_coords["Capital City"] = city_coords["Capital City"].apply(
    lambda x: x[x.find("(") + 1:x.find(")")] if "(" in x and ")" in x else x
)

In [611]:
city_coords

,Country,Capital City,Latitude,Longitude
0,Albania,Tirana,41.3275,19.8189
1,Armenia,Yerevan,40.1820,44.5146
2,Austria,Vienna,48.2064,16.3707
3,Azerbaijan,Baku,40.3777,49.8920
4,Belarus,Minsk,53.9000,27.5667
5,Belgium,Bruxelles-Brussel,50.8467,4.3499
6,Bosnia and Herzegovina,Sarajevo,43.8486,18.3564
7,Bulgaria,Sofia,42.6975,23.3242
8,Croatia,Zagreb,45.8144,15.9780
9,Cyprus,Nicosia,35.1595,33.3669


### Rename columns

In [612]:
city_coords.rename(columns={
    "Country": "country",
    "Capital City": "capital_city",
    "Latitude": "latitude",
    "Longitude": "longitude"
}, inplace=True)

In [613]:
city_coords

,country,capital_city,latitude,longitude
0,Albania,Tirana,41.3275,19.8189
1,Armenia,Yerevan,40.1820,44.5146
2,Austria,Vienna,48.2064,16.3707
3,Azerbaijan,Baku,40.3777,49.8920
4,Belarus,Minsk,53.9000,27.5667
5,Belgium,Bruxelles-Brussel,50.8467,4.3499
6,Bosnia and Herzegovina,Sarajevo,43.8486,18.3564
7,Bulgaria,Sofia,42.6975,23.3242
8,Croatia,Zagreb,45.8144,15.9780
9,Cyprus,Nicosia,35.1595,33.3669


### Add remaining values
Unfortunately, we'll have to manually add Kosovo.

In [614]:
city_coords.loc[-1] = ["Kosovo", "Pristina", 42.667542, 21.166191]
city_coords.index = city_coords.index + 1
city_coords = city_coords.sort_values(by="country")

In [615]:
city_coords

,country,capital_city,latitude,longitude
1,Albania,Tirana,41.327500,19.818900
2,Armenia,Yerevan,40.182000,44.514600
3,Austria,Vienna,48.206400,16.370700
4,Azerbaijan,Baku,40.377700,49.892000
5,Belarus,Minsk,53.900000,27.566700
6,Belgium,Bruxelles-Brussel,50.846700,4.349900
7,Bosnia and Herzegovina,Sarajevo,43.848600,18.356400
8,Bulgaria,Sofia,42.697500,23.324200
9,Croatia,Zagreb,45.814400,15.978000
10,Cyprus,Nicosia,35.159500,33.366900


### Calculate the distance, in miles and km, between cities and Moscow
Let's start by creating two columns: `moscow_lat`, and `moscow_long`, with its coordinates. Then, we need to calculate the distance, in miles and km, between each city and Moscow.


In [616]:
city_coords["moscow_lat"] = city_coords.loc[city_coords["capital_city"] == "Moscow", "latitude"].values[0]
city_coords["moscow_long"] = city_coords.loc[city_coords["capital_city"] == "Moscow", "longitude"].values[0]

In [617]:
city_coords.head()

,country,capital_city,latitude,longitude,moscow_lat,moscow_long
1,Albania,Tirana,41.3275,19.8189,55.755,37.6218
2,Armenia,Yerevan,40.1820,44.5146,55.755,37.6218
3,Austria,Vienna,48.2064,16.3707,55.755,37.6218
4,Azerbaijan,Baku,40.3777,49.8920,55.755,37.6218
5,Belarus,Minsk,53.9000,27.5667,55.755,37.6218


In [618]:
city_coords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 1 to 44
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       45 non-null     object 
 1   capital_city  45 non-null     object 
 2   latitude      45 non-null     float64
 3   longitude     45 non-null     float64
 4   moscow_lat    45 non-null     float64
 5   moscow_long   45 non-null     float64
dtypes: float64(4), object(2)
memory usage: 2.5+ KB


In [619]:
def calculate_distance(row, unit):
    """
    Calculate the distance between Moscow and a city in either kilometers or miles.
    
    Args:
        row (pd.Series): A row of the DataFrame containing latitude and longitude.
        unit (str): The unit of distance, either "km" for kilometers or "mi" for miles.
    
    Returns:
        float: The distance in the specified unit.
    """
    coords_moscow = (row["moscow_lat"], row["moscow_long"])
    coords_city = (row["latitude"], row["longitude"])
    distance = geodesic(coords_moscow, coords_city)

    return distance.km if unit == "km" else distance.mi

city_coords["distance_km"] = city_coords.apply(calculate_distance, axis=1, unit="km")
city_coords["distance_mi"] = city_coords.apply(calculate_distance, axis=1, unit="mi")


In [620]:
city_coords

,country,capital_city,latitude,longitude,moscow_lat,moscow_long,distance_km,distance_mi
1,Albania,Tirana,41.327500,19.818900,55.755,37.6218,2061.505081,1280.959870
2,Armenia,Yerevan,40.182000,44.514600,55.755,37.6218,1804.240314,1121.102955
3,Austria,Vienna,48.206400,16.370700,55.755,37.6218,1674.501261,1040.486845
4,Azerbaijan,Baku,40.377700,49.892000,55.755,37.6218,1932.401811,1200.738817
5,Belarus,Minsk,53.900000,27.566700,55.755,37.6218,677.698516,421.102335
6,Belgium,Bruxelles-Brussel,50.846700,4.349900,55.755,37.6218,2260.638611,1404.695709
7,Bosnia and Herzegovina,Sarajevo,43.848600,18.356400,55.755,37.6218,1905.407170,1183.965125
8,Bulgaria,Sofia,42.697500,23.324200,55.755,37.6218,1779.443902,1105.695179
9,Croatia,Zagreb,45.814400,15.978000,55.755,37.6218,1871.837413,1163.105845
10,Cyprus,Nicosia,35.159500,33.366900,55.755,37.6218,2311.991102,1436.604668


## Combine both datasets
It is time now to merge both datasets!

In [621]:
merged_table = sipri_joined_table.merge(
    right=city_coords,
    how="inner",
    on="country"
)

In [622]:
merged_table

,country,year,military_expenditures_usd_2022,military_expenditures_current_usd,share_GDP,share_govt,capital_city,latitude,longitude,moscow_lat,moscow_long,distance_km,distance_mi
0,Albania,1993,1.515335e+08,3.928984e+07,0.031995,NaN,Tirana,41.3275,19.8189,55.755,37.6218,2061.505081,1280.959870
1,Albania,1994,1.448474e+08,4.964950e+07,0.025478,NaN,Tirana,41.3275,19.8189,55.755,37.6218,2061.505081,1280.959870
2,Albania,1995,1.349759e+08,5.090752e+07,0.020536,NaN,Tirana,41.3275,19.8189,55.755,37.6218,2061.505081,1280.959870
3,Albania,1996,1.212103e+08,4.571336e+07,0.013790,NaN,Tirana,41.3275,19.8189,55.755,37.6218,2061.505081,1280.959870
4,Albania,1997,8.462970e+07,2.982549e+07,0.012831,0.044247,Tirana,41.3275,19.8189,55.755,37.6218,2061.505081,1280.959870
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,United Kingdom,2019,6.109445e+10,5.656760e+10,0.019853,0.051413,London,51.5085,-0.1257,55.755,37.6218,2508.674067,1558.817796
1360,United Kingdom,2020,6.206489e+10,5.833237e+10,0.021622,0.043251,London,51.5085,-0.1257,55.755,37.6218,2508.674067,1558.817796
1361,United Kingdom,2021,6.300819e+10,6.513622e+10,0.020732,0.045081,London,51.5085,-0.1257,55.755,37.6218,2508.674067,1558.817796
1362,United Kingdom,2022,6.408163e+10,6.408163e+10,0.020739,0.047094,London,51.5085,-0.1257,55.755,37.6218,2508.674067,1558.817796


# Export the final dataset
Finally! We can export the table as `output`!

In [623]:
merged_table.to_csv(f"{transformed_data}military_expenditures_final.csv", index=False, sep=',')

# Extra – Convert to Hyper using API
As an extra, let's try to use the `tableahyperapi` library to automatically generate a `.hyper` extract for our data.

In [624]:
def run_create_hyper_file_from_dataframe(df, path):
    """
    Load data from a pandas DataFrame into a new Hyper file.
    """

    print("Example – Load data from pandas DataFrame in new Hyper file")

    # 1. Load DataFrame
    data = df
    # 2. Define path where Hyper file will be saved
    path_to_database = path

    # 3. Optional process parameters.
    # These settings limit the number of log files and their size.
    process_parameters = {
        "log_file_max_count": "2", # Limit the number of log files to 2
        "log_file_size_limit": "100M" # Limit the log size to 100mb
    }

    # 4. Start the Hyper Process
    # Telemetry is set to sed usage data to Tableau
    with HyperProcess(
        telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU, parameters=process_parameters
    ) as hyper:
        
        # 5: Optional connection parameters
        # This sets the locale for the time formats to "en_US"
        connection_parameters = {"lc_time": "en_US"}

        # 6: Create a connection to the Hyper file.
        # If the file exists, it will be replaced.
        with Connection(
            endpoint=hyper.endpoint,
            database=path_to_database,
            create_mode=CreateMode.CREATE_AND_REPLACE,
            parameters=connection_parameters,
        ) as connection:
            
            # 7. Define the table schema
            table = TableDefinition(
                table_name="Europe Military Expenditures", # Table name
                columns= [
                    TableDefinition.Column(
                        "country", SqlType.text(), NOT_NULLABLE
                    ),
                    TableDefinition.Column(
                        "year", SqlType.int(), NOT_NULLABLE
                    ),
                    TableDefinition.Column(
                        "military_expenditures_usd_2022", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "military_expenditures_current_usd", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "share_GDP", SqlType.double()
                    ),
                                        TableDefinition.Column(
                        "share_govt", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "capital_city", SqlType.text()
                    ),                    
                    TableDefinition.Column(
                        "latitude", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "longitude", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "moscow_lat", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "moscow_long", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "distance_km", SqlType.double()
                    ),
                    TableDefinition.Column(
                        "distance_mi", SqlType.double()
                    ),  
                ]
            )
            
            # 8. Create the table in the hyper file
            connection.catalog.create_table(table_definition=table)

            # 9. Use the Inserter to insert data into the table.
            with Inserter(connection, table) as inserter:
                # Iterate over the DataFrame rows as tuples.
                # `itertuples` returns an iterator yielding named tuples.
                for row in df.itertuples(index=False, name=None):
                    inserter.add_row(row) # Add each row to the inserter
                inserter.execute() # Execute the insertion into the Hyper file
            
            # 10. Verify the number of rows inserted.
            row_count = connection.execute_scalar_query(
                f"SELECT COUNT(*) FROM {table.table_name}"
            )

            print(
                f"The number of rows in table {table.table_name} is {row_count}"
            )

            print("Data has been successfully inserted into the Hyper file.")
        
        # The connection is automatically closed when exiting the 'with' block.
        print("The connection to the Hyper file has been closed.")

    # The Hyper process is automatically shut down when exiting the 'with' block.
    print("The Hyper process has been shut down")

In [626]:
run_create_hyper_file_from_dataframe(df=merged_table, path=f"{refined_data}european_military_expenditures.hyper")

Example – Load data from pandas DataFrame in new Hyper file
The number of rows in table "Europe Military Expenditures" is 1364
Data has been successfully inserted into the Hyper file.
The connection to the Hyper file has been closed.
The Hyper process has been shut down


Great! Our dataset has been successfully inserted into a Hyper file, which can be directly consumed in Tableau!

# Conclusion
Wow, we went through a lot! We've managed to:
- Import our data
- Clean, pivot, and enrich our data
- Store it as a `.csv` file
- Convert it to a `.hyper` file to be used in Tableau

Our following challenges would involve creating a script for this process (which we'll do separately). For now, we move on to the analysis step! Thank you for following me so far!